# Practical session 3: Virtual biopsy of brain tumors combining magnetic resonance spectroscopy with artificial neural networks


**Authors:**  
- Anna Panfil
- Igor Czudy

**Professor:**  
Pablo Ferri Borredà

---

*Date: 16.10.2023


### Read and explore the data

In [1]:
import pandas as pd
import numpy as np 

In [2]:
df = pd.read_csv("Metabolite_concentrations_diagnosis.csv", sep=";", decimal=",", index_col=0)

df.head()

,METABOLITE 0,METABOLITE 1,METABOLITE 2,METABOLITE 3,METABOLITE 4,METABOLITE 5,METABOLITE 6,METABOLITE 7,METABOLITE 8,METABOLITE 9,METABOLITE 10,METABOLITE 11,METABOLITE 12,METABOLITE 13,METABOLITE 14,TYPE
0,0.322695,0.575024,0.742645,0.635958,0.544804,0.785336,0.885756,0.577938,0.123017,0.701689,0.731254,0.846980,3.551957,3.742832,1.719949,MENINGIOMA
1,0.457408,0.764700,0.444451,0.451300,0.388213,0.401597,0.382574,0.407632,0.715072,1.216084,1.204490,2.329079,3.179366,3.117351,1.252479,MENINGIOMA
2,1.452870,1.152910,0.647735,0.622494,0.648020,0.826787,0.797439,0.695269,0.850328,0.910488,0.914841,1.805095,2.156737,2.152553,1.081571,MENINGIOMA
3,0.157588,0.314415,0.251613,0.186498,0.100556,0.287618,0.334349,0.213802,0.305852,0.754096,0.627211,1.566407,4.274654,4.210731,1.128131,MENINGIOMA
4,0.368681,0.206993,0.211521,0.034699,0.164187,0.364590,0.340892,0.110287,0.260206,0.531801,0.496068,0.654805,4.070067,4.324002,1.350580,MENINGIOMA


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,-1], test_size=0.3, random_state=42, stratify=df.iloc[:,-1])

print(f"{X_train.shape=}")
print(f"{X_test.shape=}")


X_train.shape=(63, 15)
X_test.shape=(28, 15)


In [4]:
import ydata_profiling as ydp

# generate profiling report and save it
report = ydp.ProfileReport(pd.concat([X_train, y_train], axis=1))
report.to_file("report.html")

/home/anna/.local/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/home/anna/.conda/envs/ml/lib/python3.10/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset contains 91 examples described by 16 features – 15 metabolites and type of cancer. The data were divided to the training and test set. The training set will be described further.

There are 63 examples in this set. Each example has predictive feature – type of cancer. Classes are unbalanced. Meningtoma appears 36 times, astrocytoma – 16 and glioblastoma – 11. It's inversely proportional to the malignancy of the tumor. Therefore, it is crucial to detect the minority classes correctly. 
The metabolites ranges vary, but generally it can be said that there are real numbers mostly greater than 0 and always under 4,5. Distribution of values mostly aren't normal, maybe with except of metabolite 9 (but its a bit skewed) and 11. Some metabolites are highly correlated. Eg. metabolite 12 and 13 are highly correlated and also correlate negatively with all metabolites from 0 to 10. Simmilar situation occurs with metabolites 11 and 14. Moreover metabolite 2 quite highly correlates with 3 and 5 with 6.

### Prepare the data

In [5]:
import torch 
import torch.nn.functional as F

y_train_encoded = pd.get_dummies(y_train)
y_test_encoded = pd.get_dummies(y_test)
y_test_encoded.head()

,ASTROCYTOMA,GLIOBLASTOMA,MENINGIOMA
6,0,0,1
66,1,0,0
13,0,0,1
59,1,0,0
90,0,1,0


In [6]:
from sklearn.preprocessing import RobustScaler
import pickle

numerical_cols = X_train.columns

robust_scaler = RobustScaler(quantile_range=(5, 95))  # 95% percentile range

X_train[numerical_cols] = robust_scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = robust_scaler.transform(X_test[numerical_cols])

pickle.dump(robust_scaler, open('scaler.pkl', 'wb'))


In [7]:
from torch.utils.data import DataLoader, TensorDataset

X_train_tensor = torch.tensor(X_train.to_numpy()).float()
y_train_tensor = torch.tensor(y_train_encoded.to_numpy()).float()
X_test_tensor = torch.tensor(X_test.to_numpy()).float()
y_test_tensor = torch.tensor(y_test_encoded.to_numpy()).float()


train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

### Test various networks

In [8]:
import torch.nn as nn
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [9]:
class ComplexNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(ComplexNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [10]:
def train_model(model, batch_size, num_epochs, lr):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    criterion = nn.CrossEntropyLoss(torch.tensor([35, 15, 13]))
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        total_loss = 0
        for batch_X, batch_y in train_loader:
            # Forward pass
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss}')

    return model


In [ ]:
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

learning_rates = [0.1, 0,3, 0.5]#, 0.01, 0.001,]
batch_sizes = [4, 8, 16, 32]
num_epochs_list = [10, 20, 50, 100, 150, 200]

result = {}

input_size = X_test_tensor.shape[1]
num_classes = 3

for num_epochs in num_epochs_list:
    for lr in learning_rates:
        for batch_size in batch_sizes:
            for model_name, model in {"SimpleNN_32": SimpleNN(input_size,32, num_classes),
                                    "SimpleNN_64": SimpleNN(input_size,64, num_classes),
                                    "ComplexNN_32": ComplexNN(input_size,32, num_classes),
                                    "ComplexNN_64": ComplexNN(input_size, 64, num_classes),
                                    "SimpleNN_16": SimpleNN(input_size,16, num_classes),                          
                                    }.items():

                train_model(model, batch_size, num_epochs, lr)

                # Test the model
                test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
                model.eval()
                y_true = []
                y_pred = []
                y_scores = []
                with torch.no_grad():
                    for batch_X, batch_y in test_loader:
                        outputs = model(batch_X)
                        _, predicted = torch.max(outputs.data, 1) 
                        _, true_labels = torch.max(batch_y, 1)

                        y_true.extend(true_labels.cpu().numpy())
                        y_pred.extend(predicted.cpu().numpy())
                        probabilities = torch.softmax(outputs, dim=1)
                        y_scores.extend(probabilities.cpu().numpy())


                geometric_mean = balanced_accuracy_score(y_true, y_pred)  # użyj 'weighted' dla wieloklasowego problemu klasyfikacji
                precision = precision_score(y_true, y_pred, average='weighted')  # Oblicz precision
                recall = recall_score(y_true, y_pred, average='weighted')  # Oblicz recall
                f1 = f1_score(y_true, y_pred, average='weighted')  # Oblicz F1-score
                
                
                auc = roc_auc_score(y_true, y_scores, multi_class='ovr')  # Oblicz AUC

                print(f'Geometric Mean on the test set: {geometric_mean}')
                print(f'Precision on the test set: {precision}')
                print(f'Recall on the test set: {recall}')
                print(f'F1-score on the test set: {f1}')

                # Dodaj wynik do słownika result
                result[(model_name, num_epochs, lr, batch_size)] = {
                    'geometric_mean': geometric_mean,
                    'precision': precision,
                    'recall': recall,
                    'f1_score': f1,
                    'AUC': auc
                }

In [22]:
sorted_results = sorted(result.items(), key=lambda x: x[1]['f1_score'], reverse=True)
sorted_results[:5]

[(('SimpleNN_16', 50, 0.1, 16),
  {'geometric_mean': 0.8172619047619047,
   'precision': 0.8553921568627452,
   'recall': 0.8571428571428571,
   'f1_score': 0.8546453546453546,
   'AUC': 0.898749137336094}),
 (('SimpleNN_16', 100, 0.1, 4),
  {'geometric_mean': 0.7964285714285714,
   'precision': 0.8273809523809524,
   'recall': 0.8214285714285714,
   'f1_score': 0.8221778221778221,
   'AUC': 0.8992805999211279}),
 (('SimpleNN_64', 100, 0.1, 8),
  {'geometric_mean': 0.7964285714285714,
   'precision': 0.8273809523809524,
   'recall': 0.8214285714285714,
   'f1_score': 0.8221778221778221,
   'AUC': 0.9049630903085871}),
 (('SimpleNN_16', 100, 0.1, 16),
  {'geometric_mean': 0.7964285714285714,
   'precision': 0.8273809523809524,
   'recall': 0.8214285714285714,
   'f1_score': 0.8221778221778221,
   'AUC': 0.9130277654540078}),
 (('SimpleNN_64', 150, 0.1, 8),
  {'geometric_mean': 0.7964285714285714,
   'precision': 0.8273809523809524,
   'recall': 0.8214285714285714,
   'f1_score': 0.82217

In [25]:
best = sorted_results[0][0]
best


('SimpleNN_16', 50, 0.1, 16)

### Best model

In [26]:
# train the best model and save it
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model = SimpleNN(input_size,16, num_classes)
train_model(model, best[3], best[1], best[2])
torch.save(model, "model.pt")

Epoch [10/50], Loss: 11.246776700019836
Epoch [20/50], Loss: 3.8881739675998688
Epoch [30/50], Loss: 1.396323710680008
Epoch [40/50], Loss: 0.4803379625082016
Epoch [50/50], Loss: 0.25058419071137905
